# **Tarea 04.- Filtrado de sitios en la base de datos nacional RENAMECA**
---
## *Subcoordinación de Posgrado y Educación Continua.*
### [Instituto Mexicano de Tecnología del Agua](https://www.gob.mx/imta).<br>

<img src="Imagenes/imta_logo.png" style="height: 5em; vertical-align: middle;">
<img src="Imagenes/Mexico.jpg" style="height: 5em; vertical-align: middle;">

**Alumno: Ing. Omar Ulises Robles Pereyra** <br>
**Docente: Dr. Gabriel Ruiz Martinez** <br>

[![Open In Colab](Imagenes/colab-badge.svg)](https://colab.research.google.com/github/OmarURP/Tratamiento_Datos/blob/master/Clase_04.ipynb)

---

#### **Basado en los apuntes del Dr. Gabriel Ruiz Martinez**

**Instrucciones:**
1. Generar un archivo html que muestre en un mapa la ubicación espacial de cada uno de los sitios de muestreo y mediante el uso de "popups" se deberán visualizar los principales atributos geográficos.
2. Cargar la base de datos usando el módulo Polars.
3. Mostrar el tiempo que el programa inirtió en cargar los datos del RENAMECA, usando Polars.
4. Almacenar en un archivo tipo feather todos los parámetros que corresponden a los sitios de muestreo que se filtraron.
5. Presenta el tiempo que tarda el programa de cómputo en cargar la base da datos usando Pandas y Polars

In [1]:
# Carga de librerías necesarias
import folium
import polars as pl
import pandas as pd
import time
import os

In [2]:
# Carga de datos
datos = './Datos/03_Tarea/RESULTADOS_VERACRUZ.xlsx'
sitios = './Datos/04_Tarea/CoatzaEstaciones.xlsx'
marcador = 'c'
archivo_salida = './Resultados/04_Tarea/04_Mapa.html'
print(f'Datos cargados desde: {datos}')
print(f'Sitios cargados desde: {sitios}')
print(f'Marcador: {marcador}')
print(f'Archivo de salida: {archivo_salida}')

Datos cargados desde: ./Datos/03_Tarea/RESULTADOS_VERACRUZ.xlsx
Sitios cargados desde: ./Datos/04_Tarea/CoatzaEstaciones.xlsx
Marcador: c
Archivo de salida: ./Resultados/04_Tarea/04_Mapa.html


In [3]:
def menu_teselas(opcion):
    # Diccionario de teselas disponibles
    opciones_teselas = {
        1: 'OpenStreetMap.Mapnik',
        2: 'OpenStreetMap.HOT',
        3: 'Stamen.Terrain',
        4: 'Stamen.Toner',
        5: 'Stamen.Watercolor',
        6: 'CartoDB.Positron',
        7: 'CartoDB.DarkMatter',
        8: 'Esri.WorldStreetMap',
        9: 'Esri.WorldImagery',
        10: 'Esri.NatGeoWorldMap',
        11: 'NASAGIBS.ModisTerraTrueColorCR'
    }
    return opciones_teselas.get(opcion)

# Usar la función para seleccionar la tesela
tesela = menu_teselas(10)
print(f'Tesela a usar: {tesela}')


Tesela a usar: Esri.NatGeoWorldMap


---
### **Uso de Pandas para carga de datos** ###

In [4]:
# Inicio de cronómetro
tiempo_pandas = time.time()
print(f'Inicio de cronómetro Pandas: {tiempo_pandas:.4f}')
# Cargando hojas en un diccionario de dataframes
diccionario = pd.read_excel(datos, sheet_name=None)
# Separando por hojas el archivo
keys = list(diccionario.keys())
hoja1 = diccionario[keys[0]] # Hoja "Sitios"
hoja2 = diccionario[keys[1]] # Hoja "Resultados"
hoja3 = diccionario[keys[2]] # Hoja "Simbología"
# Ver información básica de cada hoja
for nombre, df in diccionario.items():
    print(f"\nHoja: {nombre}")
    print(f"Dimensiones: {df.shape}")
# Fin de cronómetro
final_pandas = time.time()
print("\nFin de cronómetro Pandas:", final_pandas)
tiempo_ejecucion_pandas = final_pandas - tiempo_pandas
print(f'\nTiempo de ejecución con Pandas: {tiempo_ejecucion_pandas:.2f} segundos')

Inicio de cronómetro Pandas: 1757325869.9301

Hoja: Sitios
Dimensiones: (417, 14)

Hoja: Resultados
Dimensiones: (10337, 251)

Hoja: Simbología
Dimensiones: (245, 3)

Fin de cronómetro Pandas: 1757325885.9402254

Tiempo de ejecución con Pandas: 16.01 segundos


---
### **Uso de Polars para carga de datos** ###

In [5]:
# Inicio de cronómetro
tiempo_polars = time.time()
print(f'Inicio de cronómetro Polars: {tiempo_polars:.4f}')

Inicio de cronómetro Polars: 1757325885.9501


#### **Cargando la base de datos del RENAMECA (Resultados_Veracruz)**

In [6]:
# Ver cuántas hojas tiene disponible el archivo RESLTADOS_VERACRUZ.xlsx
BaseDatos_numero_hojas = pd.ExcelFile(datos)
print(f'Número de hojas disponibles: {len(BaseDatos_numero_hojas.sheet_names)}')
# Ver cuales son los nombres de las hojas
BaseDatos_hojas = BaseDatos_numero_hojas.sheet_names
print(f'Nombres de hojas en el archivo: {BaseDatos_hojas}')

Número de hojas disponibles: 3
Nombres de hojas en el archivo: ['Sitios', 'Resultados', 'Simbología']


In [7]:
# Diccionarios de Base de datos para Polars
BaseDatos_diccionario_pl = {}
for hoja in BaseDatos_hojas:
    BaseDatos_diccionario_pl[hoja] = pl.read_excel(datos, sheet_name=hoja, infer_schema_length=0)
    print(f'Hoja cargada en Polars: {hoja} con {BaseDatos_diccionario_pl[hoja].height} filas y {BaseDatos_diccionario_pl[hoja].width} columnas')
#print(BaseDatos_diccionario_pl)

Hoja cargada en Polars: Sitios con 417 filas y 14 columnas
Hoja cargada en Polars: Resultados con 10337 filas y 251 columnas
Hoja cargada en Polars: Simbología con 245 filas y 3 columnas


#### **Cargando los sitios de interés por graficar**

In [8]:
# Ver cuántas hojas tiene disponible el archivo RESLTADOS_VERACRUZ.xlsx
Sitios_numero_hojas = pd.ExcelFile(sitios)
print(f'Número de hojas disponibles: {len(Sitios_numero_hojas.sheet_names)}')
# Ver cuales son los nombres de las hojas
Sitios_hojas = Sitios_numero_hojas.sheet_names
print(f'Nombres de hojas en el archivo: {Sitios_hojas}')

Número de hojas disponibles: 1
Nombres de hojas en el archivo: ['Sheet 1']


In [9]:
# Diccionarios de Base de datos Polars
Sitios_diccionario_pl = {}
for hoja in Sitios_hojas:
    Sitios_diccionario_pl[hoja] = pl.read_excel(sitios, sheet_name=hoja)
    print(f'Hoja cargada en Polars: {hoja} con {Sitios_diccionario_pl[hoja].height} filas y {Sitios_diccionario_pl[hoja].width} columnas')
#print(Sitios_diccionario_pl)

Hoja cargada en Polars: Sheet 1 con 691 filas y 6 columnas


In [10]:
# Fin de cronómetro
final_polars = time.time()
print("Fin de cronómetro Polars:", final_polars)
tiempo_ejecucion_polars = final_polars - tiempo_polars
print(f'Tiempo de ejecución con Polars: {tiempo_ejecucion_polars:.2f} segundos')

Fin de cronómetro Polars: 1757325888.1457632
Tiempo de ejecución con Polars: 2.20 segundos


In [11]:
# Seleccionando la columna "CLAVE SITIO" en la hoja "Sheet 1"
Clave_Sitio = Sitios_diccionario_pl["Sheet 1"].select("CLAVE_SITIO")
# Convertir a pandas para análisis sencillo
Clave_Sitio = Clave_Sitio.to_pandas()
#print(Clave_Sitio)
# Contar ocurrencias de cada sitio
print(Clave_Sitio.nunique())
print("\nConteo de sitios por clave:")
conteo_sitios = Clave_Sitio.value_counts().reset_index()
conteo_sitios.columns = ['CLAVE SITIO', 'NO MONITOREOS']
print(conteo_sitios)
# Creando lista para filtro
# Filtrando por Clave de Monitoreo
Filtro = Clave_Sitio["CLAVE_SITIO"].unique().tolist()
#print(Filtro)

CLAVE_SITIO    18
dtype: int64

Conteo de sitios por clave:
    CLAVE SITIO  NO MONITOREOS
0     OCGCE3210             42
1     OCGCE3367             42
2     OCGCE3454             42
3     OCGCE3375             42
4     OCGCE3434             42
5     OCGCE3445             42
6     OCGCE3453             42
7     OCGCE3456             42
8     OCGCE3211             41
9     OCGCE3455             41
10    OCGCE3459             41
11    OCGCE3461             41
12    OCGCE3462             41
13    OCGCE3452             39
14    OCGCE3451             39
15    OCGCE3349             38
16  OCGCE3450W1             33
17    OCGCE6474              1


In [12]:
# Seleccionando la hoja "Sitios" y convirtiendo a pandas
Sitios_df = BaseDatos_diccionario_pl["Sitios"].to_pandas()
#print(Resultados_df)
# Impriniendo las columnas disponibles
print(Sitios_df.columns)
# Filtrando el dataframe por los sitios de interés
Sitios_filtrados_df = Sitios_df[Sitios_df['CLAVE SITIO'].isin(Filtro)]
# Agregando la columna NO MONITOREOS
Sitios_filtrados_df = pd.merge(
    Sitios_filtrados_df, 
    conteo_sitios, 
    left_on='CLAVE SITIO',  # Columna en Sitios_filtrados_df
    right_on='CLAVE SITIO',  # Columna en conteo_sitios
    how='left'  # Para mantener todos los registros de Sitios_filtrados_df
)
print(f'Tamaño del dataframe filtrado: {Sitios_filtrados_df.shape}')
print(Sitios_filtrados_df[['CLAVE SITIO','LATITUD', 'LONGITUD', 'NO MONITOREOS']])

Index(['CLAVE SITIO', 'NOMBRE DEL SITIO', 'CUENCA', 'CLAVE ACUÍFERO',
       'ACUÍFERO', 'ORGANISMO CUENCA', 'DIRECCIÓN LOCAL', 'ESTADO',
       'MUNICIPIO', 'CUERPO DE AGUA', 'TIPO DE CUERPO DE AGUA',
       'SUBTIPO CUERPO AGUA', 'LATITUD', 'LONGITUD'],
      dtype='object')
Tamaño del dataframe filtrado: (18, 15)
    CLAVE SITIO    LATITUD    LONGITUD  NO MONITOREOS
0     OCGCE3210  17.424576  -94.967069             42
1     OCGCE3211  17.415814  -94.914575             41
2     OCGCE3349  17.860617   -94.65843             38
3     OCGCE3367   17.96861   -94.48034             42
4     OCGCE3375   17.88543   -94.56953             42
5     OCGCE3434   18.10664   -94.45354             42
6     OCGCE3445  17.638942  -94.750381             42
7   OCGCE3450W1   18.11742   -94.41657             33
8     OCGCE3451    17.9295   -94.55191             39
9     OCGCE3452   18.00932   -94.44534             39
10    OCGCE3453   18.01432    -94.4481             42
11    OCGCE3454    18.0141   -94.4

In [13]:
# Guardar los resultados en formato feather para acceso rápido
feather = './Resultados/04_Tarea/Sitios_Filtrados.feather'
Sitios_filtrados_df.to_feather(feather)
print(f"Datos filtrados guardados en: {feather}")

Datos filtrados guardados en: ./Resultados/04_Tarea/Sitios_Filtrados.feather


In [14]:
# Bibliotecas necesarias para cargar y visualizar datos geoespaciales
import geopandas as gpd
from folium import GeoJson

# Límites geográficos de México
lim_lat = (16.928088, 18.636030)  # (sur, norte)
lim_lon = (-95.911567, -93.678068)  # (oeste, este)

# Creación de mapa base
mapa = folium.Map(location=[(lim_lat[0] + lim_lat[1])/2, (lim_lon[0] + lim_lon[1])/2],
                 zoom_start=5,
                 tiles=tesela,
                 control_scale=True)

mapa.fit_bounds([[lim_lat[0], lim_lon[0]], [lim_lat[1], lim_lon[1]]])

# Shapefile como capa base
ruta_shapefile = "Datos/03_Tarea/Shapefile/Cuenca.shp"

# Cargar el shapefile como GeoDataFrame
gdf = gpd.read_file(ruta_shapefile)

# Verificar el sistema de coordenadas
print(f"Sistema de coordenadas original: {gdf.crs}")

# Si es necesario, transformar a WGS84 (sistema que usa Folium)
if gdf.crs != "EPSG:4326":
    gdf = gdf.to_crs(epsg=4326)
    print("Coordenadas transformadas a WGS84")

# Añadir el shapefile al mapa
GeoJson(
    data=gdf,
    style_function=lambda x: {
        'fillColor': '#3388ff',
        'color': '#004c99',
        'weight': 2,
        'fillOpacity': 0.2
    },
    tooltip=folium.GeoJsonTooltip(fields=['CVE_CUE'], labels=True),
    name="Cuenca Coatzacoalcos"
).add_to(mapa)

# Chuleando la tipografía del mapa (Respuesta generada usando L.L.M. Gemini 2.5)
for _, sitio in Sitios_filtrados_df.iterrows():
    # Crear HTML para el popup con formato mejorado y campos completos
    html = f"""
    <div style="font-family: Arial; max-width: 300px; padding: 10px;">
        <h4 style="color: #2c3e50; margin-bottom: 10px; border-bottom: 2px solid #3498db;">{sitio["NOMBRE DEL SITIO"]}</h4>
        <table style="width: 100%; border-collapse: collapse;">
            <tr style="background-color: #f2f2f2;">
                <td style="padding: 4px; font-weight: bold;">Clave sitio:</td>
                <td>{sitio["CLAVE SITIO"]}</td>
            </tr>
            <tr>
                <td style="padding: 4px; font-weight: bold;">Cuenca:</td>
                <td>{sitio["CUENCA"]}</td>
            </tr>
            <tr style="background-color: #f2f2f2;">
                <td style="padding: 4px; font-weight: bold;">Clave Acuífero:</td>
                <td>{sitio["CLAVE ACUÍFERO"]}</td>
            </tr>
            <tr>
                <td style="padding: 4px; font-weight: bold;">Acuífero:</td>
                <td>{sitio["ACUÍFERO"]}</td>
            </tr>
            <tr style="background-color: #f2f2f2;">
                <td style="padding: 4px; font-weight: bold;">Organismo Cuenca:</td>
                <td>{sitio["ORGANISMO CUENCA"]}</td>
            </tr>
            <tr>
                <td style="padding: 4px; font-weight: bold;">Dirección Local:</td>
                <td>{sitio["DIRECCIÓN LOCAL"]}</td>
            </tr>
            <tr style="background-color: #f2f2f2;">
                <td style="padding: 4px; font-weight: bold;">Estado:</td>
                <td>{sitio["ESTADO"]}</td>
            </tr>
            <tr>
                <td style="padding: 4px; font-weight: bold;">Municipio:</td>
                <td>{sitio["MUNICIPIO"]}</td>
            </tr>
            <tr style="background-color: #f2f2f2;">
                <td style="padding: 4px; font-weight: bold;">Cuerpo de agua:</td>
                <td>{sitio["CUERPO DE AGUA"]}</td>
            </tr>
            <tr>
                <td style="padding: 4px; font-weight: bold;">Tipo de cuerpo de agua:</td>
                <td>{sitio["TIPO DE CUERPO DE AGUA"]}</td>
            </tr>
            <tr style="background-color: #f2f2f2;">
                <td style="padding: 4px; font-weight: bold;">Subtipo cuerpo agua:</td>
                <td>{sitio["SUBTIPO CUERPO AGUA"]}</td>
            </tr>
            <tr>
                <td style="padding: 4px; font-weight: bold;">Coordenadas:</td>
                <td>{sitio["LATITUD"]}, {sitio["LONGITUD"]}</td>
            </tr>
            </tr>
                <tr style="background-color: #f2f2f2;">
                <td style="padding: 4px; font-weight: bold;">Monitoreos:</td>
                <td>{sitio["NO MONITOREOS"]}</td>
            </tr>
        </table>
    </div>
    """
    
    # Crear el marcador encima del shapefile
    folium.CircleMarker(
        location=(sitio["LATITUD"], sitio["LONGITUD"]),
        radius=5,
        fill_color="blue",
        fill_opacity=0.6,
        color="black",
        weight=1,
        popup=folium.Popup(html, max_width=350)
    ).add_to(mapa)

# Control de capas e impresión de resultados
folium.LayerControl().add_to(mapa)
mapa.save(archivo_salida)

Sistema de coordenadas original: EPSG:4326


In [15]:
# Mensaje de resultados impresos
print(f'Mapa guardado en: {archivo_salida}')
print(f'Se exportaron {Sitios_filtrados_df.shape[0]} sitios al mapa.')
print(f'\n La diferencia de tiempo entre Pandas y Polars es de {tiempo_ejecucion_pandas - tiempo_ejecucion_polars:.2f} segundos a favor de Polars.')

Mapa guardado en: ./Resultados/04_Tarea/04_Mapa.html
Se exportaron 18 sitios al mapa.

 La diferencia de tiempo entre Pandas y Polars es de 13.81 segundos a favor de Polars.


#### **Resultados: Mapa generado**
![Mapa](Resultados/04_Tarea/04_Mapa.png)

---
#### **Uso de `Polars` para el procesamiento de datos**

[Polars](https://www.pola.rs/) es una biblioteca de análisis de datos de alto rendimiento implementada en Rust que ofrece una interfaz en Python. Se destaca por:

1. **Rendimiento superior**: Generalmente más rápido que Pandas para operaciones en grandes conjuntos de datos.
2. **Evaluación perezosa**: Optimiza las operaciones antes de ejecutarlas.
3. **Procesamiento paralelo**: Aprovecha automáticamente múltiples núcleos del CPU.
4. **API similar a Pandas**: Facilita la transición entre ambas bibliotecas.

#### **Formato `Feather`**

El formato [Feather](https://arrow.apache.org/docs/python/feather.html) es parte del proyecto Apache Arrow y ofrece:

1. **Velocidad**: Lectura y escritura rápidas (5-10x más rápido que formatos como CSV o Excel).
2. **Eficiencia de memoria**: Almacenamiento eficiente en disco y memoria.
3. **Interoperabilidad**: Compatible con múltiples lenguajes como Python, R, Julia.
4. **Preservación de tipos**: Mantiene los tipos de datos exactos sin conversiones.

Feather es ideal para el almacenamiento intermedio de datos en flujos de trabajo de análisis de datos, especialmente cuando se trabaja con grandes conjuntos de datos.